In [7]:
import networkx as nx
import pandas as pd

path='/home/flav/gossip/flexi-pipe'

In [8]:
#Causal graph
causal_graph = nx.DiGraph([('D', 'messageReceived'),
                           ('D', 'graft'),
                           ('D', 'prune'),
                           ('D', 'iwant'),
                           ('Dlo', 'graft'),
                           ('Dhi', 'prune'),
                           ('Dscore', 'graft'),
                           ('Dscore', 'prune'),
                           ('Dlazy', 'iwant'),
                           ('Dlazy', 'ihave'),
                           ('Dout', 'graft'),
                           ('Dout', 'prune'),
                           ('gossipFactor', 'iwant'),
                           ('gossipFactor', 'ihave'),
                           ('interval', 'graft'),
                           ('interval', 'prune'),
                           ('interval', 'iwant'),
                           ('interval', 'ihave'),
                           ('topology', 'graft'),
                           ('topology', 'prune'),
                           ('topology', 'propagationTime'),
                           ('topology', 'messageReceived'),
                           #Level 1
                           ('prune', 'graft'),
                           ('prune', 'propagationTime'),
                           ('messageReceived', 'MessageOverhead'),
                           ('messageOverhead', 'bandwidth'),
                           ('iwant', 'bandwidth'),
                           ('ihave', 'bandwidth')
                         ])

In [9]:
#Import data 
data = pd.read_csv(path+'/data/Notebooks/datasets/timeSeries10_verified.csv',header=0, index_col=0)
df = pd.DataFrame(data)

df.head(10)

,identifier,experiment,type,_time,count
0,0,391.0,2.0,2023-10-04 19:25:37+00:00,1574
1,0,391.0,2.0,2023-10-04 19:25:40+00:00,1554
2,0,391.0,2.0,2023-10-04 19:25:42+00:00,236
3,0,391.0,2.0,2023-10-04 19:25:43+00:00,1540
4,0,391.0,2.0,2023-10-04 19:25:46+00:00,1535
5,0,391.0,2.0,2023-10-04 19:25:49+00:00,1547
6,0,391.0,2.0,2023-10-04 19:25:52+00:00,1525
7,0,391.0,2.0,2023-10-04 19:25:55+00:00,1555
8,0,391.0,2.0,2023-10-04 19:25:58+00:00,1522
9,0,391.0,2.0,2023-10-04 19:26:01+00:00,1534
